
*  ** Yin, Z., & Zhang, J. (2023). LSTM-based EEG emotion recognition. Biomedical Signal Processing and Control, 81, 104479. **
*  ** Li, X., Song, D., Zhang, P., et al. (2021). IEEE Access, 9, 69276-69286. **

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Input

# === 1. LOAD DATA ===
data_path = '/content/drive/MyDrive/dataset/preprocessed-data/mahnob_HCI_preprocessed_all.npy'
data = np.load(data_path, allow_pickle=True)
X_eeg = np.array([entry['eeg_data'] for entry in data])     # [samples, 5, 512]
y = np.array([entry['labels'] for entry in data])           # [samples, ...]

# === 2. (OPTIONAL) BALANCE CLASSES: for fair results
def balance_classes(X, y, label_idx):
    labels = y[:, label_idx]
    classes, counts = np.unique(labels, return_counts=True)
    max_count = np.max(counts)
    Xb, yb = [], []
    for c in classes:
        idxs = np.where(labels == c)[0]
        Xc = X[idxs]
        yc = y[idxs]
        if len(Xc) < max_count:
            X_over, y_over = resample(Xc, yc, replace=True, n_samples=max_count-len(Xc), random_state=42)
            Xc = np.concatenate([Xc, X_over])
            yc = np.concatenate([yc, y_over])
        Xb.append(Xc)
        yb.append(yc)
    return np.concatenate(Xb), np.concatenate(yb)

# (You may balance for both valence [idx=1] and then arousal [idx=2], or just one)
X_eeg_bal, y_bal = balance_classes(X_eeg, y, label_idx=1)  # Valence
X_eeg_bal, y_bal = balance_classes(X_eeg_bal, y_bal, label_idx=2)  # Then Arousal if you want

# === 3. PREPARE INPUT FOR LSTM ===
# Keras LSTM expects: [samples, time_steps, features]
# Let's feed each sample as 512 time steps, 5 features (channels)
X_lstm = np.transpose(X_eeg_bal, (0, 2, 1))  # [samples, 512, 5]

# Choose label: valence or arousal
y_valence = y_bal[:, 1]
y_arousal = y_bal[:, 2]
num_valence = int(np.max(y_valence)) + 1
num_arousal = int(np.max(y_arousal)) + 1
yv_cat = to_categorical(y_valence, num_classes=num_valence)
ya_cat = to_categorical(y_arousal, num_classes=num_arousal)

# === 4. SPLIT FOR VALENCE & AROUSAL ===
X_tr, X_te, yv_tr, yv_te, ya_tr, ya_te = train_test_split(X_lstm, yv_cat, ya_cat, test_size=0.2, random_state=42)

# === 5. LSTM-ONLY MODEL: STATE OF THE ART FOR EMOTION (e.g., Yin et al., Biomed Signal Process Control, 2023) ===
def build_lstm(num_classes):
    model = Sequential([
        Input(shape=(512, 5)),
        LSTM(128, return_sequences=True),
        BatchNormalization(),
        Dropout(0.3),
        LSTM(64),
        BatchNormalization(),
        Dropout(0.3),
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model_lstm_val = build_lstm(num_valence)
model_lstm_aro = build_lstm(num_arousal)

# === 6. TRAIN & EVALUATE: VALENCE ===
print('--- LSTM (EEG) Valence ---')
history_val = model_lstm_val.fit(
    X_tr, yv_tr,
    epochs=50,
    batch_size=128,
    validation_data=(X_te, yv_te)
)
loss_val, acc_val = model_lstm_val.evaluate(X_te, yv_te, verbose=0)
print(f"LSTM (EEG) Valence Accuracy: {acc_val*100:.2f}%")

# === 7. TRAIN & EVALUATE: AROUSAL ===
print('--- LSTM (EEG) Arousal ---')
history_aro = model_lstm_aro.fit(
    X_tr, ya_tr,
    epochs=50,
    batch_size=128,
    validation_data=(X_te, ya_te)
)
loss_aro, acc_aro = model_lstm_aro.evaluate(X_te, ya_te, verbose=0)
print(f"LSTM (EEG) Arousal Accuracy: {acc_aro*100:.2f}%")


--- LSTM (EEG) Valence ---
Epoch 1/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 317ms/step - accuracy: 0.1066 - loss: 2.6646 - val_accuracy: 0.1809 - val_loss: 2.2980
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - accuracy: 0.1943 - loss: 2.2225 - val_accuracy: 0.1809 - val_loss: 2.2946
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - accuracy: 0.2397 - loss: 2.1486 - val_accuracy: 0.1489 - val_loss: 2.2911
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - accuracy: 0.2672 - loss: 2.0476 - val_accuracy: 0.1915 - val_loss: 2.2875
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - accuracy: 0.3251 - loss: 1.8947 - val_accuracy: 0.2021 - val_loss: 2.2845
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - accuracy: 0.2699 - loss: 1.9104 - val_accuracy: 0.1915 - val_loss: 2.2823
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - accuracy: 0.3164 - loss: 1.9007 - val_accuracy: 0.1596 - val_loss: 2.2803
Epoch 8/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - accuracy: 0.3210 - loss: 1.7841 - val_accur